In [0]:
import keras
import numpy as np
import cv2
from keras.applications import resnet50
from keras.datasets import mnist
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from keras.applications.imagenet_utils import decode_predictions
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import sys
import os

Using TensorFlow backend.


In [0]:
IMG_SIZE = 28
NUM_CLASSES = 10

In [0]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

#A Method of changin grayscale to rgb

dim = (32,32)
def to_rgb(img):
  img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA) 
  img_rgb = np.asarray(np.dstack((img, img, img)), dtype=np.uint8)
  return img_rgb

rgb_list = []
for i in range(len(train_images)):
  rgb = to_rgb(train_images[i])
  rgb_list.append(rgb)

rgb_arr = np.stack([rgb_list],axis=4)
train_images = np.squeeze(rgb_arr, axis=4)

rgb_list = []
for i in range(len(test_images)):
  rgb = to_rgb(test_images[i])
  rgb_list.append(rgb)

rgb_arr = np.stack([rgb_list],axis=4)
test_images = np.squeeze(rgb_arr, axis=4)


train_images = train_images.astype('float32')
test_images = test_images.astype('float32')
train_images = train_images / 255.0
test_images = test_images / 255.0

train_labels = keras.utils.to_categorical(train_labels, NUM_CLASSES)
test_labels = keras.utils.to_categorical(test_labels, NUM_CLASSES)

print(train_images.shape)
print(test_images.shape)

(60000, 32, 32, 3)
(10000, 32, 32, 3)


In [0]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer, UpSampling2D, BatchNormalization
from keras.models import Sequential
from keras import optimizers
from keras.applications.resnet50 import ResNet50
from keras.models import Model
from skimage.transform import resize

restnet = ResNet50(include_top=False, weights='imagenet', input_shape=(32,32,3))


#x = restnet.output
x = restnet.layers[-1].output
x = Flatten()(x)
x = BatchNormalization()(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x) # was .3
x = BatchNormalization()(x)
#x = Dense(64, activation='relu')(x) # was 128
#x = Dropout(0.5)(x) # was .3
#x = BatchNormalization()(x)
prediction = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(restnet.input, output=prediction)

for layer in restnet.layers:
    layer.trainable = False

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 1s 0us/step


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Update your `Model` call to the Keras 2 API: `Model(Tensor("in..., outputs=Tensor("de...)`


In [0]:
opt = optimizers.RMSprop(lr=2e-5)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [0]:
history = model.fit(train_images, train_labels,
                    epochs=10, 
                    batch_size=8,
                    validation_data=(test_images, test_labels),
                    shuffle=True,verbose=1)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 528s 9ms/step - loss: 1.1176 - accuracy: 0.6322 - val_loss: 5.4060 - val_accuracy: 0.1013
Epoch 2/10
60000/60000 [==============================] - 528s 9ms/step - loss: 0.9838 - accuracy: 0.6758 - val_loss: 4.6498 - val_accuracy: 0.1530
Epoch 3/10
60000/60000 [==============================] - 545s 9ms/step - loss: 0.9514 - accuracy: 0.6892 - val_loss: 5.5568 - val_accuracy: 0.0989
Epoch 4/10
60000/60000 [==============================] - 537s 9ms/step - loss: 0.9397 - accuracy: 0.6926 - val_loss: 5.4136 - val_accuracy: 0.1017
Epoch 5/10
60000/60000 [==============================] - 552s 9ms/step - loss: 0.9334 - accuracy: 0.6954 - val_loss: 5.9586 - val_accuracy: 0.1474
Epoch 6/10
60000/60000 [==============================] - 561s 9ms/step - loss: 0.9272 - accuracy: 0.6982 - val_loss: 5.8597 - val_accuracy: 0.0998
Epoch 7/10
60000/60000 [==============================] - 548s